In [3]:
# 환경설정
#여름철 일반용 갑(2)전기요금 사용
Ipip install GEKKO
from gekko import GEKKO
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import matplotlib
import matplotlib.pyplot as plt
df = pd.read_excel("C:/Users/GACHON/OneDrive/바탕 화면/입력데이터_.xlsx")
m=GEKKO()
df

SyntaxError: ignored

In [ ]:
K = range(24)
dk = 1
P_L = np.array(df["Load[kWh]"],float) #부하
P_PV = np.array(df["PV_Generation[kWh]"],float) 
SMP = np.array(df["SMP[원/kWh]"],float)
TOU = np.array(df["TOU[원/kWh]"],float)

p_grid = [m.Var() for k in K] #Grid에서 가져온 전기

p_c_e1 = [m.Var() for k in K] #ESS1 충전량
p_d_e1 = [m.Var() for k in K] #ESS1 방전량
soc_e1 = [m.Var() for k in K] #ESS1의 SOC상태

u_e1 = [m.Var(integer=True) for k in K] #ESS 이진변수
u_g = [m.Var(integer=True) for k in K] #계통 이진변수

CE_ESS1 = 0.95 #ESS1 충전효율
DE_ESS1 = 0.95 #ESS1 방전효율

SOC_ESS1_init = 500 #ESS1 초기값

N1 = 15000 #계통제약

In [ ]:
#비음조건
m.Equation([p_c_e1[k] >= 0 for k in K])
m.Equation([p_d_e1[k] >= 0 for k in K])
m.Equation([soc_e1[k] >= 0 for k in K])

1. 계통 모델링

In [ ]:
m.Equation([p_grid[k] == P_L[k] + p_c_e1[k]- P_PV[k] - p_d_e1[k]*DE_ESS1  for k in K ])
m.Equation([p_grid[k]<= N1 for k in K]) #그리드 이진변수
#m.Equation([p_grid[k]<= N1 for k in K]) #그리드 이진변수
m.Equation([ 0 <= p_grid[k] for k in K])

2. ESS 모델링

In [ ]:
#ESS1의 SOE 조건
m.Equation([soc_e1[0] == SOC_ESS1_init + CE_ESS1*p_c_e1[0]- p_d_e1[0]])
m.Equation([soc_e1[k] == soc_e1[k-1] + CE_ESS1*p_c_e1[k] - p_d_e1[k] for k in range(1,23)])

#SOE 제약조건
ess_cap=1000 #[kW]
soc_max_per= ess_cap*0.95
soc_min_per= ess_cap*0.1

m.Equation([soc_e1[k] <= soc_max_per for k in K])
m.Equation([soc_min_per <= soc_e1[k] for k in K])

# 충방전 제약조건
p_c_e_per_min= ess_cap*0.01
p_c_e_per_max= ess_cap*0.5

m.Equation([u_e1[k]*p_c_e_per_min <= p_c_e1[k] for k in K ])
m.Equation([p_c_e1[k] <= p_c_e_per_max*u_e1[k] for k in K ])

m.Equation([(1-u_e1[k]) * p_c_e_per_min<=p_d_e1[k] for k in K ])
m.Equation([p_d_e1[k] <= p_c_e_per_max* (1-u_e1[k]) for k in K ])

3. 수익

In [ ]:
#revenue = sum(p_sold[k]*SMP[k] for k in K)

In [ ]:
cost = sum((P_L[k]- P_PV[k]+ p_c_e1[k]*u_e1[k] - DE_ESS1*p_d_e1[k]*(1-u_e1[k]))*TOU[k] for k in K)
#cost = sum (p_grid[k]*TOU[k] +(CE_ESS1*p_c_e1[k] + p_d_e1[k]) * 2 for k in K)

In [ ]:
m.Obj(cost)
m.options.SOLVER=1
m.solve()

apm 203.249.127.37_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          408
   Intermediates:            0
   Connections  :            0
   Equations    :          312
   Residuals    :          312
 
 Number of state variables:            408
 Number of total equations: -          311
 Number of slack variables: -          264
 ---------------------------------------
 Degrees of freedom       :           -167
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.14 NLPi:   23 Dpth:    0 Lvs:    3 Obj:  2.11E+06 Gap:       NaN
Iter:     2 I:

In [ ]:
result=pd.DataFrame({'p_c_e1': np.round(np.array(p_c_e1). flatten(),2),
                     'p_d_e1': np.round(np.array(p_d_e1). flatten(),2),
                     'p_grid': np.round(np.array(p_grid). flatten(),2),
                     'p_sold': np.round(np.array(p_sold). flatten(),2),
                     'soe_e1': np.round(np.array(soe_e1). flatten(),2),
                     'u_e1': np.round(np.array(u_e1). flatten(),2),
                     'u_g': np.round(np.array(u_g). flatten(),2)})

In [ ]:
result

In [ ]:
result.to_excel("result_1.1.xlsx")